In [49]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

base_url = "https://stackoverflow.com"
url = "https://stackoverflow.com/questions/tagged/web-scraping?tab=Votes"
s = requests.Session()
s.headers.update({'User-Agent': 'Mozilla/5.0'})

questions_info = []

for _ in range(5):
    response = s.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    questions = soup.find_all('div', class_='s-post-summary')

    for question in questions:
        q_title = question.find('h3').text.strip()
        q_link = base_url + question.find('a', class_='s-link')['href']
        q_tags = [tag.text for tag in question.find_all('a', class_='post-tag')]
        
        # Find the votes, views, and answer count
        q_votes_elem = question.select_one('.s-post-summary--stats-item__emphasized span')
        #q_votes = q_votes_elem.text if q_votes_elem is not None else 'N/A'
        
        q_views_elem = question.select_one('.s-post-summary--stats-item.is-supernova span')
        #q_views = q_views_elem.text if q_views_elem is not None else 'N/A'
        
        q_answers_elem = question.select_one('.s-post-summary--stats-item.has-answers span')
        #q_answers = q_answers_elem.text if q_answers_elem is not None else 'N/A'

        # Fetch the full question by making a request to the question's page
        q_page = s.get(q_link)
        q_soup = BeautifulSoup(q_page.text, 'html.parser')
        q_full = q_soup.find('div', class_='js-post-body').text.strip()

        # Add the question details to the list
        questions_info.append({
            'Title': q_title,
            'Link': q_link,
            'Tags': q_tags,
            'Votes': q_votes,
            'Views': q_views,
            'Answers': q_answers,
            'Full Question': q_full,
        })

        if len(questions_info) >= 100:
            break

    next_link = soup.find('a', rel='next')

    if next_link is None or len(questions_info) >= 100:
        break

    url = base_url + next_link['href']
    time.sleep(1)

df = pd.DataFrame(questions_info)
df

,Title,Link,Tags,Votes,Views,Answers,Full Question
0,How to find elements by class,https://stackoverflow.com/questions/5041008/ho...,"[python, html, web-scraping, beautifulsoup]",631,1.1m,19,I'm having trouble parsing HTML elements with ...
1,Headless Browser and scraping - solutions [clo...,https://stackoverflow.com/questions/18539491/h...,"[selenium, web-scraping, scrapy, phantomjs, ca...",377,N/A,3,Closed. This question is seeking recommendatio...
2,How do I prevent site scraping? [closed],https://stackoverflow.com/questions/3161548/ho...,"[html, web-scraping, architecture, piracy-prev...",334,136k,26,Closed. This question needs to be more focused...
3,How can I scrape a page with dynamic content (...,https://stackoverflow.com/questions/8049520/ho...,"[javascript, python, web-scraping, python-2.x]",270,449k,18,I'm trying to develop a simple web scraper. I ...
4,Scraping: SSL: CERTIFICATE_VERIFY_FAILED error...,https://stackoverflow.com/questions/50236117/s...,"[python, web-scraping, beautifulsoup, scrapy, ...",265,375k,24,I'm practicing the code from 'Web Scraping wit...
...,...,...,...,...,...,...,...
95,scrape websites with infinite scrolling,https://stackoverflow.com/questions/12519074/s...,"[python, screen-scraping, scraper]",32,N/A,3,I have written many scrapers but I am not real...
96,BeautifulSoup: what's the difference between '...,https://stackoverflow.com/questions/45494505/b...,"[python, html, web-scraping, beautifulsoup, lxml]",31,N/A,2,When using Beautiful Soup what is the differen...
97,Python selenium multiprocessing,https://stackoverflow.com/questions/53475578/p...,"[python, python-3.x, selenium, web-scraping, m...",31,N/A,3,I've written a script in python in combination...
98,scrapy: convert html string to HtmlResponse ob...,https://stackoverflow.com/questions/27323740/s...,"[python, web-scraping, scrapy]",30,N/A,3,I have a raw html string that I want to conver...


In [71]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

base_url = "https://stackoverflow.com"
url = "https://stackoverflow.com/questions/tagged/web-scraping?tab=Votes"
s = requests.Session()
s.headers.update({'User-Agent': 'Mozilla/5.0'})

questions_info = []

for _ in range(5):
    response = s.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    questions = soup.find_all('div', class_='s-post-summary')

    for question in questions:
        q_title = question.find('a', class_='s-link').text.strip()
        q_link = base_url + question.find('a', class_='s-link')['href']
        q_tags = [tag.text for tag in question.find_all('a', class_='post-tag')]

        q_votes = question.select_one('.s-post-summary--stats-item__emphasized span').text

        q_answers_span = question.select_one('.s-post-summary--stats-item.has-answers span')
        q_answers = q_answers_span.text if q_answers_span else 'N/A'

        q_views_span_supernova = question.select_one('.s-post-summary--stats-item.is-supernova span')
        q_views_span_hot = question.select_one('.s-post-summary--stats-item.is-hot span')

        if q_views_span_supernova is not None:
            q_views = q_views_span_supernova.text
        elif q_views_span_hot is not None:
            q_views = q_views_span_hot.text
        else:
            print(f"Couldn't find views for question: {q_link}")
            q_views = 'N/A'
        
        # Fetch the full question by making a request to the question's page
        q_page = s.get(q_link)
        q_soup = BeautifulSoup(q_page.text, 'html.parser')
        q_full = q_soup.find('div', class_='js-post-body').text.strip()

        # Add the question details to the list
        questions_info.append({
            'Title': q_title,
            'Link': q_link,
            'Tags': q_tags,
            'Votes': q_votes,
            'Views': q_views,
            'Answers': q_answers,
            'Full Question': q_full,
        })

        if len(questions_info) >= 100:
            break

    next_link = soup.find('a', rel='next')

    if next_link is None or len(questions_info) >= 100:
        break

    url = base_url + next_link['href']
    time.sleep(1)



In [72]:
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
questions = pd.DataFrame(questions_info)
questions

Title  \
0                                                                    How to find elements by class   
1                                               Headless Browser and scraping - solutions [closed]   
2                                                         How do I prevent site scraping? [closed]   
3                  How can I scrape a page with dynamic content (created by JavaScript) in Python?   
4                       Scraping: SSL: CERTIFICATE_VERIFY_FAILED error for http://en.wikipedia.org   
5                              How can I get the Google cache age of any URL or web page? [closed]   
6                                               How can I pass variable into an evaluate function?   
7                                                      How can I efficiently parse HTML with Java?   
8                      How to save an image locally using Python whose URL address I already know?   
9                                                                Web scraping with Python [closed]   
10                            retrieve links from web page using python and BeautifulSoup [closed]   
11   How to use Python requests to fake a browser visit a.k.a and generate User Agent? [duplicate]   
12                                                 Problem HTTP error 403 in Python 3 Web Scraping   
13                                   Scraping html tables into R data frames using the XML package   
14                                                            can we use XPath with BeautifulSoup?   
15                                     How to scrape only visible webpage text with BeautifulSoup?   
16                                   What's the best way of scraping data from a website? [closed]   
17                                                                   XPath:: Get following Sibling   
18                       What is the difference between web-crawling and web-scraping? [duplicate]   
19                                                           selenium with scrapy for dynamic page   
20                    How to scrape a website which requires login using python and beautifulsoup?   
21                                                     Using python Requests with javascript pages   
22                                                   How to run Scrapy from within a Python script   
23                                     Extracting an information from web page by machine learning   
24                                           Is it ok to scrape data from Google results? [closed]   
25                                                                              crawler vs scraper   
26                                           How to manage log in session through headless chrome?   
27                                     Selenium-Debugging: Element is not clickable at point (X,Y)   
28                                                             Converting html to text with Python   
29                            Web Scraping in a Google Chrome Extension (JavaScript + Chrome APIs)   
30                                                                          Web scraping with Java   
31                                   What should I use to open a url instead of urlopen in urllib3   
32                                  Simple jQuery selector only selects first element in Chrome..?   
33                                              Python - make a POST request using Python 3 urllib   
34                                                          How to print an exception in Python 3?   
35                                                                        Click a Button in Scrapy   
36                                     Get meta tag content property with BeautifulSoup and Python   
37                                                Using BeautifulSoup to extract text without tags   
38                                                   How to manage a 'pool' of PhantomJS instances   
39           